In [1]:
from pathlib import Path 
import pandas as pd 
import numpy as np

In [2]:
dataset_path = Path().absolute() / "Data"

In [3]:
def retrieve_data(path: Path) -> pd.DataFrame:

    return pd.read_csv(path)

In [4]:
customers = retrieve_data(dataset_path/"noahs-customers.csv")
orders = retrieve_data(dataset_path/"noahs-orders.csv")
order_items = retrieve_data(dataset_path/"noahs-orders_items.csv")
cafe_items = retrieve_data(dataset_path/"noahs-products.csv")

The below few cells are used to merge the dataframes initialized in the above cell to get information at the level of 
order id - order year - customer id - order items - customer name

In [5]:
order_particulars = order_items[['orderid', 'sku']].merge(cafe_items[['sku', 'desc']], on = "sku", how = "left")

In [6]:
order_particulars = orders[['orderid', 'customerid', 'ordered']].merge(order_particulars, on = "orderid", how = "left")

In [7]:
customers['first_name'] = customers['name'].apply(lambda x: x.split(' ')[0])
customers['last_name'] = customers['name'].apply(lambda x: x.split(' ')[-1])

In [8]:
customer_info = customers[['customerid', 'name', 'first_name', 'last_name']]

In [9]:
order_particulars = order_particulars.merge(customer_info, how = "left", on = "customerid")

In [10]:
order_particulars['year'] = order_particulars['ordered'].apply(lambda x: x.split('-')[0])

In [11]:
order_particulars = order_particulars[order_particulars['year'] == '2017']

In [12]:
possible_contractors = set()

In [13]:
for index, row in order_particulars.iterrows():

    if row['first_name'][0] == "J" and row['last_name'][0] == "P":
        possible_contractors.add(row['name'])

In [14]:
possible_contractors = list(possible_contractors)

In [15]:
JP_customers_2017 = order_particulars[order_particulars['name'].isin(possible_contractors)]

In [16]:
JP_customers_2017[JP_customers_2017['desc'].str.contains('Coffee|Bagel', case = False, na = False)]

,orderid,customerid,ordered,sku,desc,name,first_name,last_name,year
12789,7459,1475,2017-04-05 11:42:15,DLI8820,"Coffee, Drip",Joshua Peterson,Joshua,Peterson,2017
12791,7459,1475,2017-04-05 11:42:15,BKY1573,Sesame Bagel,Joshua Peterson,Joshua,Peterson,2017


In [17]:
customers[customers['customerid'] == 1475]

,customerid,name,address,citystatezip,birthdate,phone,timezone,lat,long,first_name,last_name
474,1475,Joshua Peterson,100-75 148th St,"Jamaica, NY 11435",1947-02-05,332-274-4185,America/New_York,40.70895,-73.80856,Joshua,Peterson
